In [5]:
import os
from dotenv import load_dotenv
load_dotenv("./.env")

groq_api_key = os.getenv('GROQ_API_KEY')

In [6]:
from langchain_groq import ChatGroq

model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000024DC713A510>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000024DC7142FC0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="Hi, my name Chris and I'm a AI Engineer")])
print(response)

content="Hi Chris, it's nice to meet you!  That's a really interesting field. What kind of AI work do you do?  \n\nI'm just a humble language model, but I'm always eager to learn more about what my fellow AI enthusiasts are up to. 😊  \n\n" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 21, 'total_tokens': 86, 'completion_time': 0.118181818, 'prompt_time': 0.001331649, 'queue_time': 0.072357319, 'total_time': 0.119513467}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--5a1ec726-6617-4948-9e43-8929d0ac383f-0' usage_metadata={'input_tokens': 21, 'output_tokens': 65, 'total_tokens': 86}


In [4]:
from langchain_core.messages import AIMessage

response = model.invoke([
    HumanMessage(content="Hi, my name Chris and I'm a AI Engineer"),
    AIMessage(content="Hi Chris, it's nice to meet you!  That's a really interesting field. What kind of AI work do you do?  \n\nI'm just a humble language model, but I'm always eager to learn more about what my fellow AI enthusiasts are up to. 😊  \n\n"),
    HumanMessage(content="What's my name and what's my profession")
])

print(response)

content="You told me your name is Chris and that you are an AI Engineer.  \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm all ears! 😄  \n" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 104, 'total_tokens': 150, 'completion_time': 0.083636364, 'prompt_time': 0.002767049, 'queue_time': 0.067091933, 'total_time': 0.086403413}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--16a6426f-780b-451f-b9b6-e7f46092c210-0' usage_metadata={'input_tokens': 104, 'output_tokens': 46, 'total_tokens': 150}


In [11]:
# Message history

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [9]:
config = {'configurable':{"session_id":"chat1"}}

In [11]:
response = with_message_history.invoke([
    HumanMessage(content="Hi, my name Chris and I'm a AI Engineer")
], config=config)

print(response.content)

Hi Chris, it's nice to meet you!

That's great! What kind of AI engineering work do you do? I'm always interested in learning more about what other people are working on. 🤖  

Do you have a favorite project you've worked on?  😊



In [ ]:
response = with_message_history.invoke([
    HumanMessage(content="What's my name?")
], config=config)

print(response.content)

Your name is Chris!  😊  We met just a few messages ago.  I remember!  



How can I help you today, Chris?



: 

## Prompt Template

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage


In [3]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You're a helpful assitant. Answer all the question to hte best of your knowledge"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [7]:
chain = prompt | model

In [10]:
response = chain.invoke({"messages":[HumanMessage(content="Hi my name is Chris")]})
print(response.content)

Hello Chris!  It's nice to meet you.  

I'm happy to help with any questions you have. What can I do for you today? 😊  



In [12]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [13]:
config = {"configurable":{"session_id":"chat3"}}

In [15]:
response = with_message_history.invoke([HumanMessage(content="Hi my name is Chris")], config=config)
print(response.content)

Hello Chris, it's nice to meet you! I'm happy to help with any questions you have. 

What can I do for you today? 😊  



In [18]:
## Add more variables

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You're a helpful assitant. Answer all the question to hte best of your knowledge, answer in {language} language"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [19]:
response = chain.invoke({'messages':[HumanMessage(content="Hi my name is Chris")], "language":"Spanish"})
print(response.content)

¡Hola Chris! 😊  

Es un placer conocerte. ¿Cómo puedo ayudarte hoy?  

No dudes en preguntarme lo que necesites. Haré todo lo posible para responderte de la mejor manera. 



In [20]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
    )

config = {"configurable":{"session_id":"chat4"}}

In [21]:
response = with_message_history.invoke(
    {'messages':[HumanMessage(content="Hi, I'm Chris")], "language":"Spanish"},
    config=config
)

In [23]:
print(response.content)

¡Hola Chris! Encantado de conocerte. ¿Cómo puedo ayudarte hoy?  😄 



In [24]:
response = with_message_history.invoke(
    {'messages':[HumanMessage(content="What's my name")], "language":"English"},
    config=config
)

print(response.content)

Your name is Chris!  😊  I remember that you told me. 



Is there anything else I can help you with, Chris? 



## Managing Conversation History

In [28]:
from langchain_core.messages import SystemMessage, trim_messages, AIMessage, HumanMessage

In [41]:
trimmer = trim_messages(
    max_tokens=50,
    strategy='last',
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on='human'
)

In [42]:
messages = [
    SystemMessage(content="You're a good assistant"),
    HumanMessage(content="Hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="Whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="Thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!")
]

In [ ]:
response = trimmer.invoke(messages)

In [31]:
print(response)

[SystemMessage(content="You're a good assistant", additional_kwargs={}, response_metadata={}), HumanMessage(content="Hi! I'm bob", additional_kwargs={}, response_metadata={}), AIMessage(content='hi!', additional_kwargs={}, response_metadata={}), HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}), AIMessage(content='nice', additional_kwargs={}, response_metadata={}), HumanMessage(content='Whats 2 + 2', additional_kwargs={}, response_metadata={}), AIMessage(content='4', additional_kwargs={}, response_metadata={}), HumanMessage(content='Thanks', additional_kwargs={}, response_metadata={}), AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}), HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}), AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]


In [32]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

In [43]:
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

In [39]:
response = chain.invoke({"messages": messages + [HumanMessage(content="What ice cream do I like?")], "language":"Spanish"})
print(response.content)

Como soy un modelo lingüístico, no tengo acceso a información personal como tus preferencias de helado. 

¿Cuál es tu sabor de helado favorito?  🍦😊  



In [44]:
response = chain.invoke({"messages": messages + [HumanMessage(content="What math problem did I ask?")], "language":"Spanish"})
print(response.content)

Me preguntaste: "What's 2 + 2?". 



Is there anything else I can help you with?



## Vector store and Retriever

In [46]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [49]:
import os
from dotenv import load_dotenv

load_dotenv("./.env")

groq_api_key = os.getenv('GROQ_API_KEY')
os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")

In [50]:
from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key=groq_api_key, model="llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000024DDE322F60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000024DDE322FC0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [51]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [52]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents, embeddings)

In [53]:
vectorstore.similarity_search("cat")

[Document(id='6dbd1ec9-5b61-4df1-86e6-74839bb32eec', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='82eff94d-775f-4d1a-ac83-f55d44e37b54', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [54]:
# Async Query

await vectorstore.asimilarity_search("cat")

[Document(id='6dbd1ec9-5b61-4df1-86e6-74839bb32eec', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='82eff94d-775f-4d1a-ac83-f55d44e37b54', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [55]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='6dbd1ec9-5b61-4df1-86e6-74839bb32eec', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351058602333069),
 (Document(id='82eff94d-775f-4d1a-ac83-f55d44e37b54', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740896463394165)]

## Retrievers

In [56]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

In [58]:
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

[[Document(id='6dbd1ec9-5b61-4df1-86e6-74839bb32eec', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='82eff94d-775f-4d1a-ac83-f55d44e37b54', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [59]:
retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={"k":1}
)

retriever.batch(["cat","dog"])


[[Document(id='6dbd1ec9-5b61-4df1-86e6-74839bb32eec', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='82eff94d-775f-4d1a-ac83-f55d44e37b54', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [60]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question usign the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages(['Human', message])

In [61]:
rag_chain = {"context": retriever, "question":RunnablePassthrough()} | prompt | llm

In [63]:
response = rag_chain.invoke("tell me about dogs")
print(response.content)

Dogs are great companions, known for their loyalty and friendliness.
